***NB:*** *Denne notebooken er i utvikling, og ment som eksempel på hvordan brukere kan jobbe programmatisk med Nasjonalbiblioteket korpus av nettavis-tekster. Notebooken lar deg:*
- *bygge og visualisere korpus*
- *hente konkordanser,*
- *beregne relativfrekvens for kollokasjoner.*

*Har du spørsmål? Les gjerne [overordnet informasjon om nettavis-korpuset](https://www.nb.no/samlingen/nettarkivet/forskning/korpus-med-nettaviser/) eller send oss en e-post på [nettarkivet@nb.no](mailto:nettarkivet@nb.no).*

___

# 0. Importer `dhlab` for python

Før vi begynner må vi importere nødvendige python-pakker. Dersom du ikke har installert `dhlab` allerede må du avkommentere den første linja og fjerne `# ` foran `!pip install ...`

In [ ]:
# !pip install -U dhlab (fjern `# ` for å installere siste versjon av python-pakka for dhlab
import dhlab as dh
import dhlab.nbtext as nb
from dhlab import Corpus, totals, Collocations, Ngram
import plotly.express as px
import numpy as np

___

# 1. Korpusanalyse

Et korpus betyr en samling av tekster. I denne sammenhengen skal vi lage et korpus med tekster fra norske nettaviser.

I `dhlab` brukes Python-klassen `Corpus` til å representere en liste med metadata om hvert dokument. Her finner du metadata om publikasjonstittel, språk, dato for innhøsting, domenenavn, osv. Hver tekst i korpuset har også en unik `dhlabid`, DHlabs persistente URN.

Hvis du ønsker en oversikt over tekstenes ulike attributter, slik de eksponeres gjennom APIet, kan du lese mer om [Korpus med nettaviser](https://www.nb.no/samlingen/nettarkivet/forskning/korpus-med-nettaviser/#hvilke-skjema-attributter-kan-jeg-benytte-mot-apiet?).

## 1.1 Bygge korpus

La oss bygge et korpus!

Kodecellen under bygger et korpus med tekster fra NRK som inneholder ord som begynner på "vindmølle*".

`doctype="nettavis"` angir at det er nettavis-tekst vi vil jobbe med. `title="NRK"` avgrenser til tekster fra NRK, og `fulltext="vindmølle*"` avgrenser til tekster som inneholder ord som begynner på vindmølle.

In [ ]:
corpus = dh.Corpus(doctype="nettavis", fulltext="vindmølle*", limit=100000)
corpus

## 1.2 Innsikt og visualisering

Før du går videre og analyserer teksten ønsker du kanskje innsikt i korpuset du har bygget? La oss visualisere hvordan tekstene fordeler seg etter år, publikasjoner og språk.

### 1.2.1 Publikasjoner i korpus (tree map)

Vise distribusjon av tekster per publikasjon, og visualiserer fordelingen i et tree map.

In [ ]:
telle_titler = corpus['title'].value_counts()

fig = px.treemap(
    path=[telle_titler.index],
    values=telle_titler.values,
    title='Fordeling av tekster etter publikasjonsnavn',
    hover_data={'Antall tekster': telle_titler.values}
)

fig.show()


### 1.2.2 Fordeling av tekster over tid

In [ ]:
hoestetid = pd.to_datetime(corpus['timestamp'], format='%Y%m%d')

# Generer tilfeldige verdier for spredning langs y-aksen
y_spredning = np.random.uniform(0, 1, size=len(hoestetid))

# Legg til jitter (tilfeldig justering langs y-aksen) på høstetidspunktene
x_jitter = hoestetid + pd.to_timedelta(np.random.uniform(-2, 2, size=len(hoestetid)), unit='D')

# Lager scatter plot med jitter for bedre spredning
fig = px.scatter(
    x=x_jitter,  # Høstetidspunkter med jitter på x-aksen
    y=y_spredning,  # Tilfeldig spredning på y-aksen
    labels={'x': 'Høstetidspunkt', 'y': 'Spredning'},
    title='Tekster etter høstedato',
    opacity=0.5,  # Gjør punktene delvis gjennomsiktige
    size_max=5  # Øk punktstørrelsen
)

# Vis diagrammet
fig.show()


### 1.2.3 Fordeling av språk/målform

In [ ]:
# Tell antall forekomster av hvert språk (langs-kode)
språkfordeling = corpus['langs'].value_counts()

# Lag et kakediagram basert på språkfordelingen
fig = px.pie(
    values=språkfordeling.values,  # Antall forekomster av hvert språk
    names=språkfordeling.index,    # Språkkoder (ISO 639-2)
    title='Fordeling av språk/målform',
    labels={'names': 'Språk/målform', 'values': 'Antall forekomster'}
)

# Vis diagrammet
fig.show()


## 1.3 Eksportere korpus

Hvis du vil lagre resultatene kan du eksportere dem i Excel og/eller JSONL format. Husk å endre filnavnet og ev. filsti!

In [ ]:
# Eksporter korpus til Excel (endre filnavn og filsti)
corpus.frame.to_excel('./korpus-vindmølle.xlsx', index=False)

# Eksportere korpus til JSONL (endre filnavn og filsti)
corpus.frame.to_json('./korpus-vindmølle.jsonl', orient='records', lines=True)

___

## 2. Konkordanser

Når du har bygget korpuset kan du hente ut ulike informasjon om korpuset.

Kodecellen under henter ut konkordanser, det vil si et tekstvindu med inntil 12 ord før/etter et søkeord. Søkeordet fremheves i fet skrift.

La oss be om konkordanser for ordet "vindmølle".

In [ ]:
conc_windmill = corpus.conc(words="vindmølle*")
conc_windmill.show()

### Eksportere konkordanser

Hvis du vil lagre resultatene kan du eksportere dem i Excel og/eller JSONL format. Husk å endre filnavnet og ev. filsti!

In [ ]:
# Eksportere konkordanser til Excel
conc_windmill.frame.to_excel('./konkordanser-vindmølle.xlsx', index=False)

# Eksportere konkordanser til JSONL
conc_windmill.frame.to_json('./konkordanser-vindmølle.jsonl', orient='records', lines=True)

___

## 3. Kollokasjoner og ordfrekvenser
Kollokasjoner er ordpar som forekommer sammen. Ved å telle kollokasjoner for et gitt ord kan vi si noe om frekvensen for hvor hyppig/sjelden ulike ord forekommer sammen med hverandre.

En del ord alltid vil forekomme hyppig i alle tekster, slik som `og`, `han` og `hun`. For å finne ord som er signifikante innen en gitt kontekst kan vi beregne en relativfrekvens, der vi sammenlikner frekvensen i vårt eget korpus med et allment referansekorpus.

Cellen under lister ordene med høyest relativfrekvens, gitt et nøkkelord i korpuset.

In [ ]:
tot = totals(50000) # Henter referansekorpus for å sammenlikne relativfrekvens
coll = corpus.coll("vindmøller").frame.sort_values(by="counts", ascending=False) # Teller kollokasjoner for nøkkelord i korpuset
(coll.counts / tot.freq).sort_values(ascending = False).head(20) # Finner relativfrekvens i korpuset, sammenliknet med referansekorpuset